In [1]:
import numpy as np
from pyscf import gto, scf, fci

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for spinorbitals
def ONERDMFT_hartree_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals
def ONERDMFT_Umrigar_hartree_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            if not(a == b):
                for mu in range(0,M):
                    for nu in range(0,M):
                        for kappa in range(0,M):
                            for lamda in range(0,M):
                                energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [5]:
# for spinorbitals, I can only simulate orthogonality by excluding \alpha \beta, \beta \alpha from the sum since I know the first set of M are alpha and the second set of M are beta
def ONERDMFT_Umrigar_exchange_corelation_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M//2):
        for b in range(0,M//2):
            if not(a == b):
                for mu in range(0,M):
                    for nu in range(0,M):
                        for kappa in range(0,M):
                            for lamda in range(0,M):
                                energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  range(M//2,M):
        for b in range(M//2,M):
            if not(a == b):
                for mu in range(0,M):
                    for nu in range(0,M):
                        for kappa in range(0,M):
                            for lamda in range(0,M):
                                energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [6]:
# for spinorbitals, I can only simulate orthogonality by excluding \alpha \beta, \beta \alpha from the sum since I know the first set of M are alpha and the second set of M are beta
def ONERDMFT_exchange_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  range(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [7]:
# for spinorbitals, I can only simulate orthogonality by excluding \alpha \beta, \beta \alpha from the sum since I know the first set of M are alpha and the second set of M are beta
def ONERDMFT_Mueller_exchange_corelation_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  range(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [8]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" 
#mol.basis = "ccpvdz"
#mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

print(eri.shape)

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

C = mf.mo_coeff
h = mf.get_hcore()

N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
occu, naturalC = np.linalg.eigh(gamma)

(2, 2, 2, 2)
converged SCF energy = -2.85516042615445


In [9]:
# need to first make gamma and then expand into blockdiagonal so "fake" alpha and "beta" are seperated in this fashion
CE = expand_matrix(C)
gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
occu_aa, naturalC_aa = np.linalg.eigh(gamma)
occu = np.append(occu_aa, occu_aa)
naturalCE = expand_matrix(naturalC_aa)
naturalCTT = np.matmul(CE,naturalCE)

In [10]:
for i, n  in enumerate(occu):
    if n < 0:
        occu[i] = 0 
print(occu)

[0. 1. 0. 1.]


In [11]:
#print(naturalCTT)
E_H = ONERDMFT_hartree_energy(eri, naturalCTT, occu)
E_U = ONERDMFT_Umrigar_hartree_energy(eri, naturalCTT, occu)
E_x = ONERDMFT_exchange_energy(eri, naturalCTT, occu)
E_xc = ONERDMFT_Umrigar_exchange_corelation_energy(eri, naturalCTT, occu)
print(f"Based on expanded HF density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Umrigar Hartree: {E_U}\nUmrigar Exchangecorrelation: {E_xc}\nUmrigar Hartree + Exchangecorrelation: {E_U + E_xc}")

Based on expanded HF density matrix:
Hartree Energy: 2.053814337750754
Exchange Energy: -1.0269071688753777
Hartree + Exchange 1.0269071688753764
Umrigar Hartree: 1.0269071688753777
Umrigar Exchangecorrelation: -0.0
Umrigar Hartree + Exchangecorrelation: 1.0269071688753777


In [12]:
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
FCIgamma = fci.direct_spin1.make_rdm1(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIgamma = FCIgamma/2.0
FCIoccu, FCInaturalC = np.linalg.eigh(FCIgamma)
FCInaturalCTT = np.matmul(C,FCInaturalC)
FCInaturalCTTE = expand_matrix(FCInaturalCTT)
FCIoccuE = np.append(FCIoccu,FCIoccu)
#print(FCInaturalCTTE)

In [13]:
print(FCIoccu)

[0.00432351 0.99567649]


In [14]:
print(f"E_FCI {e} E_HF {mf.e_tot} E_c {e-mf.e_tot}")

E_FCI -2.8701621389008225 E_HF -2.8551604261544465 E_c -0.015001712746375961


In [15]:
E_H = ONERDMFT_hartree_energy(eri, FCInaturalCTTE, FCIoccuE)
E_U = ONERDMFT_Umrigar_hartree_energy(eri, FCInaturalCTTE, FCIoccuE)
E_x = ONERDMFT_exchange_energy(eri, FCInaturalCTTE, FCIoccuE)
E_xc = ONERDMFT_Umrigar_exchange_corelation_energy(eri, FCInaturalCTTE, FCIoccuE)
E_c = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_U + E_xc) - mf.e_tot
print(f"Based on expanded FCI density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Umrigar Hartree: {E_U}\nUmrigar Exchange-correlation: {E_xc}\nUmrigar Hartree + Exchange-correlation: {E_U + E_xc}")
print(f"Umrigar corelation energy {E_c}")

Based on expanded FCI density matrix:
Hartree Energy: 2.0459980008682868
Exchange Energy: -1.0175709437305762
Hartree + Exchange 1.0284270571377105
Umrigar Hartree: 1.0303892422503858
Umrigar Exchange-correlation: -0.029906289493692548
Umrigar Hartree + Exchange-correlation: 1.0004829527566932
Umrigar corelation energy -0.00793890739350811


In [16]:
h = mf.get_hcore()
2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T))))+E_U+E_xc

-2.8630993335479547

In [17]:
print(np.trace(np.matmul(h,P)))
print(np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))))

-1.9410337975149115
-1.931791143152324


In [18]:

E_H = ONERDMFT_hartree_energy(eri, FCInaturalCTTE, FCIoccuE)
E_U = ONERDMFT_hartree_energy(eri, FCInaturalCTTE, FCIoccuE)
E_x = ONERDMFT_exchange_energy(eri, FCInaturalCTTE, FCIoccuE)
E_xc = ONERDMFT_Mueller_exchange_corelation_energy(eri, FCInaturalCTTE, FCIoccuE)
E_c = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_U + E_xc) - mf.e_tot
print(f"Based on expanded FCI density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Mueller Hartree: {E_U}\nMueller Exchange-correlation: {E_xc}\nMueller Hartree + Exchange-correlation: {E_U + E_xc}")
print(f"Mueller corelation energy {E_c}")

Based on expanded FCI density matrix:
Hartree Energy: 2.0459980008682868
Exchange Energy: -1.0175709437305762
Hartree + Exchange 1.0284270571377105
Mueller Hartree: 2.0459980008682868
Mueller Exchange-correlation: -1.0532327263912111
Mueller Hartree + Exchange-correlation: 0.9927652744770756
Mueller corelation energy -0.01565658567312589


In [19]:
2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T))))+E_U+E_xc

-2.8708170118275724